# Importance Sampling

## Introduction to Importance Sampling

아래의 토끼는 배경에서 반사된 빛을 받아서 다시 재반사(reflect)하여 카메라에 투영된 이미지의 모습니다. <br>

![Illumination Integral Illustration](images/impsam_rabbit.jpg)

3D rendering으로 나온 토끼의 이미지처럼, 특정 방향 $ L_i(\mathbf{u}) $ 으로부터 들어오는 빛을 받아서, 카메라 $ \mathbf{v} $ 방향으로 재반사 하기 위해서는, Bidirectional reflectance distribution function (BRDF) 라는 material function $ f $ 를 사용합니다.  전체 반사되는 빛 $ L_0(\mathbf{v}) $ 의 양을 계산하기 위해서는 모든 각각의 방향 $ \mathbf{u} $ 으로부터 오는 모든 빛을 합 하거나 또는 integration해야 합니다. 공식은 다음과 같습니다.

$$ L_0(\mathbf{v}) = \int_H L_i(\mathbf{u}) f(\mathbf{u}, \mathbf{v}) \cos \theta_u \ du $$

공식을 자세하게 알 필요는 없습니다. <br>
포인트는 모든 방향에서는 오는 빛을 계산하여 반사되는 빛을 계산하여 적분하는것은 계산의 양이 너무나 많기 때문에 할 수 없는 방법입니다.<br>
따라서 uniform distribution으로 랜덤으로 들어오는 빛을 samples로 integral을 계산합니다. 샘플들의 평균은 해당 integral의 approximation과도 같습니다.

![Illumination Integral Illustration](images/impsam_light.jpg)


<span style="color:#cc3333">
**만약 integrated function이 어떻게 작동하는줄 대략적으로 알고 있다면**</span>, <br>
Uniform ramdom directions로부터 integral을 approximation하는 것은 좋은 방법이 아닙니다.<br>
**예를 들어서 카메라에 반사되는 빛을 구하기 위해서, 사물에 닿는 모든 빛을 구하는게 아니라, 바로 거울 처럼 반사되는 지점에서 오는 빛을 samples로 사용하는 것이 좋을 것입니다.** 왜냐하면 대부분의 카메라에 반사되는 빛은 해당 방향으로부터 빛이 오기 때문입니다.

수학적으로 표현하기 위해서, probability density function (PDF)를 사용하여 샘플링을 위한 최적의 방향을 정하게 됩니다.<br>
PDF는 normalized function이며, PDF함수의 전체 도메인에 대한 integral의 값은 1이고, 샘플링에 가장 중요한 지점은 peaks로 나타나게 됩니다.

## No Prior Knowledge on the Integrated Function

위에서 저런 가설이 사용가능한 이유는 Integrated function에 대해서 알고 있기 때문입니다. <br>
하지만 **대부분의 경우에는 integrated function에 대해서 사전에 지식이 없는 경우가 대부분이며, 어느 부분이 중요한지 알아서 샘플링은 불가능 합니다.**

여기서 Variance Reduction과 상충되게 됩니다.<br>
Variance Reduction 은 integrated function에 대해서 사전에 알고 있어야 하지만 현실은 대부분의 경우 모른다는 것입니다. <br>

가장 이상적인 상황은 다음과 같습니다.<br>
Integrand를 non-constant function에서 constant function으로 만들어주면 됩니다.<br>
Constant function으로 만든다는 뜻은 variance은 0으로 만들며 approximation은 항상 동일한 값을 얻는다는 뜻입니다.<br>
이는 Monte Carlo Integration의 목표가 variance를 최대한 작게 만들고, samples은 최대한 적은 양을 사용하는 것과도 부합합니다.<br>
아래의 그림처럼 constant function이 되면 uniform distribution으로 samples을 얻을 수 있게 됩니다.

![constant-function](images/impsam_uniform.png)

물론 이는 가장 이상적인 상황일때 입니다. 실제로 이런 일은 일어나기 쉽지 않습니다.


## Convert Non-Constant Function to Constant Function

함수를 자기자신과 나누어 버리면 항상 결과값은 1이 나오게 됩니다.<br>
예를 들어서 $ f(0)=2 $ 일때  $ \frac{f(0)}{2} = 1 $ 이 되고,  <br>
$ f(2)=0.5 $ 일때 $ \frac{f(2)}{0.5} = 1 $ 이 되게 됩니다.

![function f(x) is divided by itset](images/impsam_divided_by_self.png)

General Monte Carlo integration $ \langle F^N \rangle = \frac{1}{N} \sum^{N-1}_{i=0} \frac{f(x_i)}{pdf(x_i)} $ 에서 $ pdf(x_i) $ 부분을 $ pdf(x_i) = cf(x_i) $ 바꿔서줄 수 있습니다. <br>
(이때 조건은 가장 이상적인 상황으로서 $ pdf(x_i) $ 는 integral과 정확히 또는 매우 유사하게 비례한다고 가정한다. <br>
따라서 2번째 그림처럼 위치는 다르지만 비율은 동일하기 때문에 $ f(x) = cf'(x) $ 가 된다)


$$ \require{enclose} Y_i = \frac{f(X_i)}{pdf(X_i)} =  \frac{ \enclose{updiagonalstrike}{f(X_i)}}{ c \enclose{updiagonalstrike}{f(X_i)}}  = \frac{1}{c} $$

각각의  $ Y_i $ 는 동일한 값을 리턴하기 때문에, 전체적으로 variance도 0입니다.<br>

c를 유도하는 방법은 PDF가 integrate하면 1이 되는 점을 이용합니다.<br>
<span style="color:#777777">
$ pdf(X_i) = cf(X_i) $ 이므로.. $ \int cf(X_i) = 1 $ 이 됩니다.<br>
상수에 대한 곱은 intgration rule에서 밖으로 빠질수 있으므로 $ c \int f(X_i) = 1 $ 가 됩니다.
</span>

$$ c = \frac{1}{\int f(x)\ dx} $$

해당 공식은 불행하게도 integral $ f(x) $ 를 연산해야지만 pdf에서 사용되는 normalization constant $ c $ 를 얻을수 있다는 것을 보여줍니다.



주의할점은 이렇게 사용하게 위해서는 $ f(x) $ 가 무엇인지 알고 있어야 하며, 따라서 **모든 경우에 적용이 되지 않는다는 것**입니다.<br>
아래 그림에서 <span style="color:blue;">파란색은 intgrand function</span> 이고 <span style="color:red;">빨간색은 pdf</span>를 나타냅니다.

![Importance Sampling to work](images/impsam_pdf.png)

만약 integrand function의 shape에 대해서 전혀 모른다면 그냥 uniform distribution으로 가는게 좋습니다.<br>
물론 pdf와 intgrand가 유사한 분포를 갖고 있는것보다는 좋지 않겠지만.. 잘못 선택하여 전혀 다른 분포를 갖은 pdf 를 사용하는 것 보다는 낫습니다.

# Example

다음의 integral를 Monte Carlo Integration을 사용하여 sin함수를 approximate합니다.

$$ \begin{align} 
F &= \int^{\pi/2}_{0} sin(x)\ dx \\
&= \left[ - \cos \left( x \right) \right]^{\frac{\pi}{2}}_0  \\
&= - cos\left( \frac{\pi}{2} \right) - (- cos(0))  \\
&= 1
\end{align}$$

Uniform probability distribution $ p(x) = \frac{2}{\pi x} $ 그리고 $ p'(x) = \frac{8x}{\pi^2} $ 2개를 사용합니다.<br>
아래 그림에서 보듯이 두번째 PDF가 uniform probability distribution보다 integrand의 shape에 더 유사합니다.<br>
위에서 배운 이론대로라면 uniform probability distribution보다 두번째 PDF가 variance를 더 줄여줍니다.

![](images/impsam_example.png)

첫번째  Uniform distribution에는 다음의 estimator를 사용합니다.

$$ \langle F^N \rangle = \frac{\pi/2}{N} \sum^{N-1}_{i=0} sin(X_i) $$

$ X_i $ 는 uniform distribution으로 가져오게됩니다. <br>
두번째 PDF

In [3]:
import numpy as np


x = np.arange(0, np.pi/2, 0.001)


## 결론 

**integral $ f(X) $ 에 대해서 알고 있다면**<br>
  - integral $ f(X) $ 와 유사한 pdf를 사용하여 variance를 낮춘다. 
  
  
**integral $ f(X) $ 에 대해서 모르고 있다면**<br>
  - 전혀 알수 없다면 uniform distribution을 사용한다. 
  

  
 